In [1]:
#!pip install pandasql

In [2]:
#import library
import pandas as pd
import pandasql
import seaborn as sns
import matplotlib.pyplot as plt

from pandasql import sqldf 
sql_run = lambda q: sqldf(q, globals())

In [3]:
#import dataset
order_item = pd.read_csv('olist_order_items_dataset.csv', sep = ',', encoding = 'latin-1')
order_reviews = pd.read_csv('olist_order_reviews_dataset.csv', sep = ',', encoding = 'latin-1')
order_dataset = pd.read_csv('olist_orders_dataset.csv', sep = ',', encoding = 'latin-1')
product_dataset = pd.read_csv('olist_products_dataset.csv', sep = ',', encoding = 'latin-1')
product_category = pd.read_csv('product_category_name.csv', sep = ',', encoding = 'latin-1')

In [4]:
#export dataframe to csv
#df = sql_run(query)
#df.to_csv('dua.txt', index=False, sep='\t')

# Key Objective Analysis

In [5]:
# 1: average yearly order 
query = """
WITH
    sales_over_year AS(
SELECT
    DISTINCT seller_id,
    CAST(shipping_limit_date AS DATETIME) as year,
    ROUND(SUM (price)) AS Sales,
    SUM(DISTINCT order_item_id) AS number_of_order
FROM order_item
GROUP BY 1,2
)
SELECT 
    seller_id,
    year,
    ROUND((Sales - LAG(Sales) OVER(ORDER BY Year ASC)) / LAG(Sales) OVER(ORDER BY Year ASC) * 100,2) AS growth_rate,
    ROUND((number_of_order - LAG(number_of_order) OVER(ORDER BY Year ASC)) / LAG(number_of_order) OVER(ORDER BY Year ASC) * 100,2) AS growth_order_quantity
FROM sales_over_year 
ORDER BY 3 DESC


"""

sql_run(query)

,seller_id,year,growth_rate,growth_order_quantity
0,7c67e1448b00f6e969d365cea6b010ab,2017,315136.67,5400.0
1,850f4f8af5ea87287ac68de36e29107f,2017,145865.00,200.0
2,1f50f920176fa81dab994f9023523100,2018,107506.67,7700.0
3,7681ef142fd2c19048da7430856b5588,2018,106373.33,900.0
4,34d1ca11b242c0fee2c834ae8d788566,2017,105575.00,200.0
...,...,...,...,...
4310,6561d6bf844e464b4019442692b40e02,2018,-99.94,0.0
4311,edb58a1390adf273840030a3d6253829,2018,-99.94,0.0
4312,46ef1a25845fc051c2998d878354810c,2017,-99.95,0.0
4313,7cb946b1ad19faebe28c3e86c2794ec3,2017,-99.97,0.0


In [6]:
#2: average shopping cart size
query = """
SELECT 
    DISTINCT(seller_id),
    AVG(order_item_id) as product_quantity,
    AVG(price) as sales
FROM order_item
GROUP BY 1
ORDER BY 2 DESC
"""
sql_run(query)

,seller_id,product_quantity,sales
0,0b36063d5818f81ccb94b54adfaebbf5,8.000000,51.000000
1,2709af9587499e95e803a6498a5a56e9,5.510638,27.224681
2,1fa2d3def6adfa70e58c276bb64fe5bb,5.000000,6.900000
3,f326006815956455b2859abd58fe7e39,4.966667,104.905333
4,c394e193cda3b4225ff2094d32184849,4.750000,284.990000
...,...,...,...
3090,00d8b143d12632bad99c0ad66ad52825,1.000000,86.000000
3091,00ab3eff1b5192e5f1a63bcecfee11c8,1.000000,98.000000
3092,003554e2dce176b5555353e4f3555ac8,1.000000,120.000000
3093,001e6ad469a905060d959994f1b41e4f,1.000000,250.000000


In [7]:
#4: Order Defect Rate (ODR)
query = """
SELECT 
    order_item.seller_id, 
    AVG(order_reviews.review_score) as review_score,
    COUNT(order_reviews.order_id) as number_of_order
FROM order_item
  INNER JOIN order_reviews
  ON order_reviews.order_id = order_item.order_id
GROUP BY 1
ORDER BY 3 DESC
"""

sql_run(query)

,seller_id,review_score,number_of_order
0,6560211a19b47992c3666cc44a7e94c0,3.909406,2020
1,4a3ca9315b744ce9f8e9374361493884,3.803931,1984
2,1f50f920176fa81dab994f9023523100,3.982402,1932
3,cc419e0650a3c5ba77189a1882b7556a,4.069575,1811
4,da8622b14eb17ae2831f4ac5b9dab84a,4.071429,1568
...,...,...,...
3085,04ee0ec01589969663ba5967c0e0bdc0,5.000000,1
3086,00d8b143d12632bad99c0ad66ad52825,5.000000,1
3087,00ab3eff1b5192e5f1a63bcecfee11c8,5.000000,1
3088,003554e2dce176b5555353e4f3555ac8,5.000000,1


In [8]:
#seller and products for sale
query ="""
SELECT
    order_item.seller_id,
    product_dataset.product_category_name as product
from order_item
    inner join product_dataset
    on order_item.product_id=product_dataset.product_id
"""
sql_run(query)

,seller_id,product
0,48436dade18ac8b2bce089ec2a041202,cool_stuff
1,dd7ddc04e1b6c2c614352b383efe2d36,pet_shop
2,5b51032eddd242adc84c38acab88f23d,moveis_decoracao
3,9d7a1d34a5052409006425275ba1c2b4,perfumaria
4,df560393f3a51e74553ab94004ba5c87,ferramentas_jardim
...,...,...
112645,b8bc237ba3788b23da09c0f1f3a3288c,utilidades_domesticas
112646,f3c38ab652836d21de61fb8314b69182,informatica_acessorios
112647,c3cfdc648177fdbbbb35635a37472c53,esporte_lazer
112648,2b3e4a2a3ea8e01938cabda2a3e5cc79,informatica_acessorios


# Further Analysis

In [9]:
#based on consumers
query ="""
WITH 
product as(
SELECT
    product_dataset.product_category_name as products,
    order_item.product_id as product_id,
    COUNT(order_dataset.customer_id) as number_of_consumer
from order_item
    inner join product_dataset on order_item.product_id=product_dataset.product_id
    inner join order_dataset on order_item.order_id = order_dataset.order_id
GROUP BY 1,2
ORDER BY 3 desc
LIMIT 20)

select 
    DISTINCT products, number_of_consumer
FROM product
WHERE products != 'None'
group by 1
ORDER BY 2 DESC


"""
sql_run(query)

,products,number_of_consumer
0,moveis_decoracao,527
1,cama_mesa_banho,488
2,ferramentas_jardim,484
3,informatica_acessorios,343
4,relogios_presentes,323
5,beleza_saude,281
6,utilidades_domesticas,183
7,casa_conforto,165


In [12]:
#based on quantity
query ="""
WITH 
product as(
SELECT
    product_dataset.product_category_name as products,
    order_item.product_id as product_id,
    SUM(order_item.order_item_id) as product_quantity
from order_item
    inner join product_dataset on order_item.product_id=product_dataset.product_id
    inner join order_dataset on order_item.order_id = order_dataset.order_id
GROUP BY 1,2
ORDER BY 3 desc
LIMIT 20)

select 
    DISTINCT products, product_quantity
FROM product
WHERE products != 'None'
group by 1
ORDER BY 2 DESC


"""
sql_run(query)

,products,product_quantity
0,ferramentas_jardim,793
1,moveis_decoracao,640
2,cama_mesa_banho,542
3,informatica_acessorios,369
4,relogios_presentes,367
5,beleza_saude,300
6,automotivo,210


In [13]:
#based on sum of price
query ="""
WITH 
product as(
SELECT
    product_dataset.product_category_name as products,
    order_item.product_id as product_id,
    SUM(order_item.price) as sum_price
from order_item
    inner join product_dataset on order_item.product_id=product_dataset.product_id
    inner join order_dataset on order_item.order_id = order_dataset.order_id
GROUP BY 1,2
ORDER BY 3 desc
LIMIT 20)

select 
    DISTINCT products, sum_price
FROM product
WHERE products != 'None'
group by 1
ORDER BY 2 DESC


"""
sql_run(query)

,products,sum_price
0,beleza_saude,63885.00
1,pcs,48899.34
2,informatica_acessorios,47214.51
3,cama_mesa_banho,43025.56
4,bebes,38907.32
5,cool_stuff,37733.90
6,relogios_presentes,37683.42
7,moveis_decoracao,37608.90
8,ferramentas_jardim,26577.22
9,instrumentos_musicais,25034.00
